# Full Summary Dashboard

Comprehensive decomposition summary for all lenders:

1. **Multi-Lender Summary** - Aggregate and breakdown waterfalls (YoY and MoM)
2. **Per-Lender Volume Decomposition** - 3-column grid of booking waterfalls
3. **Per-Lender Penetration Charts** - 3-column grids of penetration analysis

All charts saved to `outputs/Summary/` and `outputs/{Lender}/` folders.

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
import math

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Import calculators
from lmdi_decomposition_calculator import (
    calculate_decomposition,
    calculate_multi_lender_decomposition
)
from lmdi_penetration_calculator import (
    calculate_penetration_decomposition,
    print_penetration_decomposition
)
from utils import is_non_financed_lender

# Import visualization functions
from visualization_summary import _create_aggregate_waterfall
from visualization_lender import create_lender_aggregate_waterfall
from visualization_penetration import (
    create_penetration_chart_net_all_inclusive,
    create_penetration_chart_net_volume_inclusive,
    _create_net_all_inclusive_waterfall,
    _create_net_volume_inclusive_waterfall
)

print("Modules loaded successfully!")

---
## User Configuration

**Specify the analysis date below.** The notebook will automatically:
- Calculate YoY (Year-over-Year) and MoM (Month-over-Month) comparisons
- Generate charts for ALL lenders in the dataset

In [ ]:
# ============================================================
# USER INPUT: Specify your analysis date here
# ============================================================

ANALYSIS_DATE = '2024-06-01'  # Format: 'YYYY-MM-01' (first of month)

# ============================================================

In [ ]:
# Load data
data_path = Path.cwd().parent / 'data' / 'funnel_data_2d_mock_with_nonfinanced.csv'
df = pd.read_csv(data_path)
df['month_begin_date'] = pd.to_datetime(df['month_begin_date'])

# Parse analysis date and calculate comparison dates
analysis_date = pd.to_datetime(ANALYSIS_DATE)
yoy_date = analysis_date - relativedelta(years=1)
mom_date = analysis_date - relativedelta(months=1)

# Get available dates and lenders
available_dates = df['month_begin_date'].unique()
all_lenders = [l for l in df['lender'].unique() if not is_non_financed_lender(l)]
all_lenders = sorted(all_lenders)

print(f"Data loaded: {len(df)} rows")
print(f"Date range: {df['month_begin_date'].min().date()} to {df['month_begin_date'].max().date()}")
print(f"Financed lenders: {all_lenders}")
print()
print("=" * 60)
print(f"Analysis Date:   {analysis_date.date()}")
print(f"YoY Comparison:  {yoy_date.date()} -> {analysis_date.date()}")
print(f"MoM Comparison:  {mom_date.date()} -> {analysis_date.date()}")
print("=" * 60)

# Validate dates
if analysis_date not in available_dates:
    raise ValueError(f"Date '{analysis_date.date()}' not found in data.")
yoy_available = yoy_date in available_dates
mom_available = mom_date in available_dates
if not yoy_available:
    print(f"WARNING: YoY date {yoy_date.date()} not in data - YoY analysis will be skipped")
if not mom_available:
    print(f"WARNING: MoM date {mom_date.date()} not in data - MoM analysis will be skipped")

In [ ]:
# Create output directories
base_output_dir = Path.cwd().parent / 'outputs'
summary_dir = base_output_dir / 'Summary'
summary_dir.mkdir(parents=True, exist_ok=True)

# Create per-lender directories
lender_dirs = {}
for lender in all_lenders:
    lender_dir = base_output_dir / lender
    lender_dir.mkdir(parents=True, exist_ok=True)
    lender_dirs[lender] = lender_dir

print(f"Output directories created:")
print(f"  Summary: {summary_dir}")
for lender, ldir in lender_dirs.items():
    print(f"  {lender}: {ldir}")

---
## Helper Functions

In [ ]:
def create_lender_grid(lenders, create_chart_func, title_prefix, n_cols=3, figsize_per_chart=(12, 8)):
    """
    Create a grid of charts for multiple lenders.
    
    Args:
        lenders: List of lender names
        create_chart_func: Function that takes (ax, lender) and creates a chart
        title_prefix: Prefix for the overall figure title
        n_cols: Number of columns in the grid (default 3)
        figsize_per_chart: Size of each chart in the grid
    
    Returns:
        matplotlib Figure
    """
    n_lenders = len(lenders)
    n_rows = math.ceil(n_lenders / n_cols)
    
    fig_width = figsize_per_chart[0] * n_cols
    fig_height = figsize_per_chart[1] * n_rows
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height))
    
    # Flatten axes for easy iteration
    if n_rows == 1 and n_cols == 1:
        axes = np.array([axes])
    axes = axes.flatten() if n_rows > 1 or n_cols > 1 else axes
    
    # Create charts for each lender
    for i, lender in enumerate(lenders):
        create_chart_func(axes[i], lender)
    
    # Hide unused subplots (fill with blank space)
    for i in range(n_lenders, len(axes)):
        axes[i].axis('off')
    
    fig.suptitle(title_prefix, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    
    return fig

---
## Pre-calculate All Results

In [ ]:
# Pre-calculate all decomposition results
yoy_results = {}  # lender -> DecompositionResults
mom_results = {}
yoy_pen_results = {}  # lender -> PenetrationResults
mom_pen_results = {}

print("Calculating decomposition results...")
print()

# YoY calculations
if yoy_available:
    print("YoY Calculations:")
    for lender in all_lenders:
        print(f"  Processing {lender}...")
        try:
            yoy_results[lender] = calculate_decomposition(
                df=df, date_a=str(yoy_date.date()), date_b=str(analysis_date.date()), lender=lender
            )
            yoy_pen_results[lender] = calculate_penetration_decomposition(
                df=df, date_a=str(yoy_date.date()), date_b=str(analysis_date.date()), lender=lender
            )
        except Exception as e:
            print(f"    WARNING: Failed for {lender}: {e}")

# MoM calculations
if mom_available:
    print("\nMoM Calculations:")
    for lender in all_lenders:
        print(f"  Processing {lender}...")
        try:
            mom_results[lender] = calculate_decomposition(
                df=df, date_a=str(mom_date.date()), date_b=str(analysis_date.date()), lender=lender
            )
            mom_pen_results[lender] = calculate_penetration_decomposition(
                df=df, date_a=str(mom_date.date()), date_b=str(analysis_date.date()), lender=lender
            )
        except Exception as e:
            print(f"    WARNING: Failed for {lender}: {e}")

# Multi-lender calculations
print("\nMulti-Lender Calculations:")
multi_yoy_results = None
multi_mom_results = None

if yoy_available:
    print("  YoY Multi-Lender...")
    multi_yoy_results = calculate_multi_lender_decomposition(
        df=df, date_a=str(yoy_date.date()), date_b=str(analysis_date.date())
    )

if mom_available:
    print("  MoM Multi-Lender...")
    multi_mom_results = calculate_multi_lender_decomposition(
        df=df, date_a=str(mom_date.date()), date_b=str(analysis_date.date())
    )

print("\nAll calculations complete!")

---
## 1. Multi-Lender Summary Charts (YoY)

In [ ]:
# YoY Multi-Lender Simple Aggregate
if multi_yoy_results:
    fig, ax = plt.subplots(figsize=(14, 8))
    
    _create_aggregate_waterfall(
        ax=ax,
        summary=multi_yoy_results.aggregate_summary,
        period_1_bks=multi_yoy_results.metadata['aggregate_period_1_bookings'],
        period_2_bks=multi_yoy_results.metadata['aggregate_period_2_bookings'],
        title=f'Multi-Lender YoY Aggregate: {yoy_date.date()} -> {analysis_date.date()}'
    )
    
    plt.tight_layout()
    output_file = summary_dir / 'yoy_multi_lender_aggregate.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    plt.show()

In [ ]:
# YoY Multi-Lender Breakdown (stacked by lender)
if multi_yoy_results:
    fig = create_lender_aggregate_waterfall(
        lender_summaries=multi_yoy_results.lender_summaries,
        aggregate_summary=multi_yoy_results.aggregate_summary,
        date_a=str(yoy_date.date()),
        date_b=str(analysis_date.date()),
        period_1_bks=multi_yoy_results.metadata['aggregate_period_1_bookings'],
        period_2_bks=multi_yoy_results.metadata['aggregate_period_2_bookings']
    )
    
    output_file = summary_dir / 'yoy_multi_lender_breakdown.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    plt.show()

---
## 2. Multi-Lender Summary Charts (MoM)

In [ ]:
# MoM Multi-Lender Simple Aggregate
if multi_mom_results:
    fig, ax = plt.subplots(figsize=(14, 8))
    
    _create_aggregate_waterfall(
        ax=ax,
        summary=multi_mom_results.aggregate_summary,
        period_1_bks=multi_mom_results.metadata['aggregate_period_1_bookings'],
        period_2_bks=multi_mom_results.metadata['aggregate_period_2_bookings'],
        title=f'Multi-Lender MoM Aggregate: {mom_date.date()} -> {analysis_date.date()}'
    )
    
    plt.tight_layout()
    output_file = summary_dir / 'mom_multi_lender_aggregate.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    plt.show()

In [ ]:
# MoM Multi-Lender Breakdown (stacked by lender)
if multi_mom_results:
    fig = create_lender_aggregate_waterfall(
        lender_summaries=multi_mom_results.lender_summaries,
        aggregate_summary=multi_mom_results.aggregate_summary,
        date_a=str(mom_date.date()),
        date_b=str(analysis_date.date()),
        period_1_bks=multi_mom_results.metadata['aggregate_period_1_bookings'],
        period_2_bks=multi_mom_results.metadata['aggregate_period_2_bookings']
    )
    
    output_file = summary_dir / 'mom_multi_lender_breakdown.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    plt.show()

---
## 3. Per-Lender Volume Decomposition Grid (YoY)

In [ ]:
# YoY Volume Grid
if yoy_available and yoy_results:
    lenders_with_results = [l for l in all_lenders if l in yoy_results]
    
    def create_yoy_volume_chart(ax, lender):
        results = yoy_results[lender]
        meta = results.metadata
        _create_aggregate_waterfall(
            ax=ax,
            summary=results.summary,
            period_1_bks=meta['period_1_total_bookings'],
            period_2_bks=meta['period_2_total_bookings'],
            title=f'{lender} YoY: {yoy_date.date()} -> {analysis_date.date()}'
        )
    
    fig = create_lender_grid(
        lenders=lenders_with_results,
        create_chart_func=create_yoy_volume_chart,
        title_prefix=f'YoY Booking Decomposition by Lender: {yoy_date.date()} -> {analysis_date.date()}',
        n_cols=3,
        figsize_per_chart=(10, 7)
    )
    
    # Save grid to Summary folder
    output_file = summary_dir / 'yoy_volume_by_lender_grid.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    # Also save individual charts to lender folders
    for lender in lenders_with_results:
        fig_individual, ax = plt.subplots(figsize=(14, 8))
        create_yoy_volume_chart(ax, lender)
        plt.tight_layout()
        individual_file = lender_dirs[lender] / f'yoy_volume_waterfall.png'
        fig_individual.savefig(individual_file, dpi=150, bbox_inches='tight', facecolor='white')
        plt.close(fig_individual)
        print(f"Saved: {individual_file}")
    
    plt.show()

---
## 4. Per-Lender Volume Decomposition Grid (MoM)

In [ ]:
# MoM Volume Grid
if mom_available and mom_results:
    lenders_with_results = [l for l in all_lenders if l in mom_results]
    
    def create_mom_volume_chart(ax, lender):
        results = mom_results[lender]
        meta = results.metadata
        _create_aggregate_waterfall(
            ax=ax,
            summary=results.summary,
            period_1_bks=meta['period_1_total_bookings'],
            period_2_bks=meta['period_2_total_bookings'],
            title=f'{lender} MoM: {mom_date.date()} -> {analysis_date.date()}'
        )
    
    fig = create_lender_grid(
        lenders=lenders_with_results,
        create_chart_func=create_mom_volume_chart,
        title_prefix=f'MoM Booking Decomposition by Lender: {mom_date.date()} -> {analysis_date.date()}',
        n_cols=3,
        figsize_per_chart=(10, 7)
    )
    
    # Save grid to Summary folder
    output_file = summary_dir / 'mom_volume_by_lender_grid.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    # Also save individual charts to lender folders
    for lender in lenders_with_results:
        fig_individual, ax = plt.subplots(figsize=(14, 8))
        create_mom_volume_chart(ax, lender)
        plt.tight_layout()
        individual_file = lender_dirs[lender] / f'mom_volume_waterfall.png'
        fig_individual.savefig(individual_file, dpi=150, bbox_inches='tight', facecolor='white')
        plt.close(fig_individual)
        print(f"Saved: {individual_file}")
    
    plt.show()

---
## 5. Per-Lender Penetration - Net All Inclusive Grid (YoY)

In [ ]:
# YoY Penetration Net All Inclusive Grid
if yoy_available and yoy_pen_results:
    lenders_with_results = [l for l in all_lenders if l in yoy_pen_results]
    n_lenders = len(lenders_with_results)
    n_cols = 3
    n_rows = math.ceil(n_lenders / n_cols)
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(12*n_cols, 8*n_rows))
    axes = axes.flatten() if n_rows > 1 or n_cols > 1 else [axes]
    
    for i, lender in enumerate(lenders_with_results):
        ax = axes[i]
        pen_result = yoy_pen_results[lender]
        
        # Convert penetration to bps for the visualization function
        period_1_pen = pen_result.metadata['period_1_penetration'] * 100 * 100  # to bps
        period_2_pen = pen_result.metadata['period_2_penetration'] * 100 * 100  # to bps
        delta_bps = pen_result.metadata['delta_penetration_bps']
        
        # Use the internal function to draw on the grid axis
        _create_net_all_inclusive_waterfall(
            ax=ax,
            summary=pen_result.summary,
            period_1_pen=period_1_pen,
            period_2_pen=period_2_pen,
            title=f'{lender} YoY: {period_1_pen/100:.1f}% -> {period_2_pen/100:.1f}% ({delta_bps:+.0f} bps)'
        )
    
    # Hide unused subplots
    for i in range(n_lenders, len(axes)):
        axes[i].axis('off')
    
    fig.suptitle(f'YoY Penetration (Net All Inclusive) by Lender: {yoy_date.date()} -> {analysis_date.date()}', 
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    
    output_file = summary_dir / 'yoy_penetration_net_inclusive_by_lender_grid.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    # Save individual charts
    for lender in lenders_with_results:
        fig_individual = create_penetration_chart_net_all_inclusive(
            summary=yoy_pen_results[lender].summary,
            metadata=yoy_pen_results[lender].metadata,
            lender=lender
        )
        individual_file = lender_dirs[lender] / f'yoy_penetration_net_all_inclusive.png'
        fig_individual.savefig(individual_file, dpi=150, bbox_inches='tight', facecolor='white')
        plt.close(fig_individual)
        print(f"Saved: {individual_file}")
    
    plt.show()

---
## 6. Per-Lender Penetration - Net All Inclusive Grid (MoM)

In [ ]:
# MoM Penetration Net All Inclusive Grid
if mom_available and mom_pen_results:
    lenders_with_results = [l for l in all_lenders if l in mom_pen_results]
    n_lenders = len(lenders_with_results)
    n_cols = 3
    n_rows = math.ceil(n_lenders / n_cols)
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(12*n_cols, 8*n_rows))
    axes = axes.flatten() if n_rows > 1 or n_cols > 1 else [axes]
    
    for i, lender in enumerate(lenders_with_results):
        ax = axes[i]
        pen_result = mom_pen_results[lender]
        
        period_1_pen = pen_result.metadata['period_1_penetration'] * 100 * 100
        period_2_pen = pen_result.metadata['period_2_penetration'] * 100 * 100
        delta_bps = pen_result.metadata['delta_penetration_bps']
        
        _create_net_all_inclusive_waterfall(
            ax=ax,
            summary=pen_result.summary,
            period_1_pen=period_1_pen,
            period_2_pen=period_2_pen,
            title=f'{lender} MoM: {period_1_pen/100:.1f}% -> {period_2_pen/100:.1f}% ({delta_bps:+.0f} bps)'
        )
    
    for i in range(n_lenders, len(axes)):
        axes[i].axis('off')
    
    fig.suptitle(f'MoM Penetration (Net All Inclusive) by Lender: {mom_date.date()} -> {analysis_date.date()}', 
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    
    output_file = summary_dir / 'mom_penetration_net_inclusive_by_lender_grid.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    for lender in lenders_with_results:
        fig_individual = create_penetration_chart_net_all_inclusive(
            summary=mom_pen_results[lender].summary,
            metadata=mom_pen_results[lender].metadata,
            lender=lender
        )
        individual_file = lender_dirs[lender] / f'mom_penetration_net_all_inclusive.png'
        fig_individual.savefig(individual_file, dpi=150, bbox_inches='tight', facecolor='white')
        plt.close(fig_individual)
        print(f"Saved: {individual_file}")
    
    plt.show()

---
## 7. Per-Lender Penetration - Net Volume Inclusive Grid (YoY)

In [ ]:
# YoY Penetration Net Volume Inclusive Grid
if yoy_available and yoy_pen_results:
    lenders_with_results = [l for l in all_lenders if l in yoy_pen_results]
    n_lenders = len(lenders_with_results)
    n_cols = 3
    n_rows = math.ceil(n_lenders / n_cols)
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(14*n_cols, 8*n_rows))
    axes = axes.flatten() if n_rows > 1 or n_cols > 1 else [axes]
    
    for i, lender in enumerate(lenders_with_results):
        ax = axes[i]
        pen_result = yoy_pen_results[lender]
        
        period_1_pen = pen_result.metadata['period_1_penetration'] * 100 * 100
        period_2_pen = pen_result.metadata['period_2_penetration'] * 100 * 100
        delta_bps = pen_result.metadata['delta_penetration_bps']
        
        _create_net_volume_inclusive_waterfall(
            ax=ax,
            summary=pen_result.summary,
            metadata=pen_result.metadata,
            period_1_pen=period_1_pen,
            period_2_pen=period_2_pen,
            title=f'{lender} YoY: {period_1_pen/100:.1f}% -> {period_2_pen/100:.1f}% ({delta_bps:+.0f} bps)'
        )
    
    for i in range(n_lenders, len(axes)):
        axes[i].axis('off')
    
    fig.suptitle(f'YoY Penetration (Net Volume Inclusive) by Lender: {yoy_date.date()} -> {analysis_date.date()}', 
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    
    output_file = summary_dir / 'yoy_penetration_net_volume_by_lender_grid.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    # Save individual charts
    for lender in lenders_with_results:
        fig_individual = create_penetration_chart_net_volume_inclusive(
            summary=yoy_pen_results[lender].summary,
            metadata=yoy_pen_results[lender].metadata,
            lender=lender
        )
        individual_file = lender_dirs[lender] / f'yoy_penetration_net_volume_inclusive.png'
        fig_individual.savefig(individual_file, dpi=150, bbox_inches='tight', facecolor='white')
        plt.close(fig_individual)
        print(f"Saved: {individual_file}")
    
    plt.show()

---
## 8. Per-Lender Penetration - Net Volume Inclusive Grid (MoM)

In [ ]:
# MoM Penetration Net Volume Inclusive Grid
if mom_available and mom_pen_results:
    lenders_with_results = [l for l in all_lenders if l in mom_pen_results]
    n_lenders = len(lenders_with_results)
    n_cols = 3
    n_rows = math.ceil(n_lenders / n_cols)
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(14*n_cols, 8*n_rows))
    axes = axes.flatten() if n_rows > 1 or n_cols > 1 else [axes]
    
    for i, lender in enumerate(lenders_with_results):
        ax = axes[i]
        pen_result = mom_pen_results[lender]
        
        period_1_pen = pen_result.metadata['period_1_penetration'] * 100 * 100
        period_2_pen = pen_result.metadata['period_2_penetration'] * 100 * 100
        delta_bps = pen_result.metadata['delta_penetration_bps']
        
        _create_net_volume_inclusive_waterfall(
            ax=ax,
            summary=pen_result.summary,
            metadata=pen_result.metadata,
            period_1_pen=period_1_pen,
            period_2_pen=period_2_pen,
            title=f'{lender} MoM: {period_1_pen/100:.1f}% -> {period_2_pen/100:.1f}% ({delta_bps:+.0f} bps)'
        )
    
    for i in range(n_lenders, len(axes)):
        axes[i].axis('off')
    
    fig.suptitle(f'MoM Penetration (Net Volume Inclusive) by Lender: {mom_date.date()} -> {analysis_date.date()}', 
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    
    output_file = summary_dir / 'mom_penetration_net_volume_by_lender_grid.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    for lender in lenders_with_results:
        fig_individual = create_penetration_chart_net_volume_inclusive(
            summary=mom_pen_results[lender].summary,
            metadata=mom_pen_results[lender].metadata,
            lender=lender
        )
        individual_file = lender_dirs[lender] / f'mom_penetration_net_volume_inclusive.png'
        fig_individual.savefig(individual_file, dpi=150, bbox_inches='tight', facecolor='white')
        plt.close(fig_individual)
        print(f"Saved: {individual_file}")
    
    plt.show()

---
## Summary of Saved Files

In [ ]:
# List all saved files
print("\n" + "=" * 80)
print("SAVED FILES SUMMARY")
print("=" * 80)

# Summary folder
print(f"\n[Summary Folder: {summary_dir}]")
print("-" * 60)
for f in sorted(summary_dir.glob('*.png')):
    size_kb = f.stat().st_size / 1024
    print(f"  {f.name:<50} ({size_kb:,.0f} KB)")

# Per-lender folders
for lender in all_lenders:
    lender_dir = lender_dirs[lender]
    files = list(lender_dir.glob('*.png'))
    if files:
        print(f"\n[{lender} Folder: {lender_dir}]")
        print("-" * 60)
        for f in sorted(files):
            size_kb = f.stat().st_size / 1024
            print(f"  {f.name:<50} ({size_kb:,.0f} KB)")

# Count totals
total_summary = len(list(summary_dir.glob('*.png')))
total_lender = sum(len(list(lender_dirs[l].glob('*.png'))) for l in all_lenders)
print("\n" + "=" * 80)
print(f"TOTAL: {total_summary} summary charts + {total_lender} lender-specific charts = {total_summary + total_lender} files")
print("=" * 80)